# Purpose
Design an amplifier that can be used as the basis for a later Q-enhanced LNA design, similar to the one described in [Episode 1, Part 3 (of 3) in Radio Design 401, by MegawattKS](https://www.youtube.com/watch?v=43wQmYeqt44&t=1282s)

# Requirements
* 20dB gain at 98MHz
* 20MHz bandwidth, i.e. the whole FM band (88-108MHz)
* 50ohm input impedance (no matching network needed)
* 7V supply voltage
* Common-base stage followed by common-emitter (aka emitter-follower) stage

# Design
## Common-base stage
### Calculate I_c
Input impedance for a CB-stage is approximately 1/gm, where gm is the transconductance of the transistor. The transconductance is given by gm = Ic/Vt, where Ic is the collector current and Vt is the thermal voltage (approximately 26mV at room temperature).  From that, we can calculate the collector current needed to achieve the desired input impedance.
$$R_{in} = \frac{1}{g_{m}},g_{m}=\frac{I_{C}}{V_{T}}\Rightarrow I_{C} = \frac{V_{T}}{R_{in}}$$

In [102]:
# Calculat the required collector current for an input impedance of 50 ohm
V_T = 26e-3  # Thermal voltage at room temperature in volts
R_in = 50  # Input impedance in ohms
I_c = V_T / R_in
print(f"Required collector current (I_c): {I_c:.2e} A")

Required collector current (I_c): 5.20e-04 A


### Calculate R_C
From that, we can calculate the collector resistor value needed for the desired gain. The voltage gain of a common-base stage is given by:
$$A_{Vloaded} = \frac{A_{Vno-load}}{2}$$
$$A_{Vno-load} = g_{m}*R_{C}$$
$$R_{C}=\frac{2*A_{Vloaded}}{g_{m}}$$
Where A_Vunloaded is the desired voltage gain of the CB-stage. The CC-stage has a voltage gain of approximately 1, so the overall voltage gain of the amplifier is approximately equal to the voltage gain of the CB-stage.

In [103]:
G_T_dB = 20  # total gain in dB
G_T = 10 ** (G_T_dB / 20)  # Total voltage gain in linear scale
A_Vunloaded = 2* G_T # Unloaded voltage gain in volts
g_m = I_c / V_T  # Transconductance in S (Siemens)
R_C = A_Vunloaded / g_m
print(f"Collector resistance (R_C): {R_C:.2f} ohm")

Collector resistance (R_C): 1000.00 ohm


### Calculate biasing network

In [104]:
def calc_bias(Ic, Vbe, V_E, hfe):
    # Calculate biasing resistors
    Re = V_E / Ic # Emitter resistor
    print("\tEmitter resistor: {:.2f} ohm".format(Re))
    Ib = Ic / hfe # Base current
    V_R2 = Vbe + V_E # Voltage over R2
    I_R2 = 12 * Ib # Current through R2 for stable biasing
    R2 = V_R2 / I_R2 # Base resistor
    print("\tBottom bias resistor: {:.2f} ohm".format(R2))
    V_R1 = Vcc - V_R2 # Voltage over R1
    I_R1 = I_R2 + Ib # Current through R1
    R1 = V_R1 / I_R1 # Top bias resistor
    print("\tTop bias resistor: {:.2f} ohm".format(R1))
    return R1, R2, Re

def stability_check(R1, R2, Re, hfe):
    R_th = R1 * R2 / (R1 + R2) # Thevenin resistance

    # Bowick "Transistor Biasing" (ISBN: 978-0-7506-8518-4): As a practical rule of thumb for stable designs, the ratio R_th/Re should be less than 10.
    if(R_th / Re) < 10:
        print("\tStability check: OK")
    else:
        print("\tStability check: NOT OK")

    S = (hfe + 1) * (Re + R_th) / ((hfe+1) * Re + R_th) # Stability factor
    print("\tStability factor: {:.2f}".format(S))
    if S < 10:
        print("\tGood biasing")
    elif S < 20:
        print("\tAcceptable biasing")
    else:
        print("\tUnstable biasing")

Vcc = 7 # Supply voltage
Vbe = 0.8 # Base-emitter voltage
V_E = 1.5 # Assumed voltage over emitter resistor
hfe = 100 # Current gain

[R1, R2, Re] = calc_bias(I_c, Vbe, V_E, hfe)

stability_check(R1, R2, Re, hfe)

	Emitter resistor: 2884.62 ohm
	Bottom bias resistor: 36858.97 ohm
	Top bias resistor: 69526.63 ohm
	Stability check: OK
	Stability factor: 8.64
	Good biasing


We have now enough info to draw the first schematic. The DC-operating point analysis can be used to check our calculations.

<figure>
<img src="./doc/DC-operating_point.png" width="80%"/>
<figcaption>DC-operating point</figcaption>
</figure>

## Common-collector stage

Let's start off by using the same DC-biasing scheme as the common-base stage.  Two transistors will be used.  One will be used for feedback, while the other will be used as output buffer.  They will share the same DC-biasing stage.

## Image filter

In [105]:
import math 
def parallel_RLC(Rp, B, f_0):
    ω = 2 * math.pi * f_0
    Q = f_0 / B
    print("\tQuality factor Q = {:.2f}".format(Q))
    X_0 = Rp / Q
    L = X_0 / ω
    print("\tCalculated L = {:.2e} H".format(L))
    C = 1 / (ω * X_0)
    print("\tCalculated C = {:.2e} F".format(C))
    B = f_0 / Q
    print("\tCalculated B = {:.2e} Hz".format(B))
    return L, C, B

f = 98e6 # 98 MHz, center frequency of the FM broadcast band
B = 20e6 # 20 MHz, bandwidth (88-108 MHz)
L, C, B = parallel_RLC(R_C, B, f)

	Quality factor Q = 4.90
	Calculated L = 3.31e-07 H
	Calculated C = 7.96e-12 F
	Calculated B = 2.00e+07 Hz


Adding these values in the simulation resulted in an f_0 of 77.5MHz and a Q of 4.47.

In [106]:
f_real = 77.5e6 # 77.5 MHz
Q_real = 4.47
L = 330e-9 # 330 nH
ω = 2 * math.pi * f_real
X_0 = ω * L
Rp_real = X_0 * Q_real
print("\tCalculated Rp = {:.2e} ohm".format(Rp_real))
R_parasitic  = R_C * Rp_real / (R_C - Rp_real)
print("\tCalculated R_parasitic = {:.2e} ohm".format(R_parasitic))

# Now we can calculate the needed R_C for an equivalent R_p of 1000 ohm
R_p = 1000  # Target parallel resistance
R_C = R_parasitic * R_p / (R_parasitic - R_p)
print("\tCalculated R_C = {:.2e} ohm".format(R_C))

	Calculated Rp = 7.18e+02 ohm
	Calculated R_parasitic = 2.55e+03 ohm
	Calculated R_C = 1.65e+03 ohm


So, the new R_C is 1600ohm.

In [107]:
L, C, B = parallel_RLC(R_C, B, f)

	Quality factor Q = 4.90
	Calculated L = 5.45e-07 H
	Calculated C = 4.84e-12 F
	Calculated B = 2.00e+07 Hz


The problem with this is that needed capacitor is smaller than the parasitic capacitance (even without taking into account the parasitic capacitance of the PCB).  

Let's keep the R_C of 1kohm and live with the reduced gain.  Using L = 222nH (WE 744918222) and C = 6.7pF, gives us center frequency of 98MHz, a peak gain of 18.8dB and a Q of 4.87.

We might also stock up on smaller value inductors.  These will needed bigger capacitors, but the Q will be higher.

<figure>
<img src="./doc/service_band_gain.png" width="80%"/>
<figcaption>Service band gain and input match</figcaption>
</figure>

<figure>
<img src="./doc/S-parameter_simulation.png" width="80%"/>
<figcaption>S-parameter simulation for 10MHz to 500MHz</figcaption>
</figure>